## Model training

In [ ]:
import face_recognition
import pickle
import os

def encode_faces(image_paths):
    known_encodings = []
    known_names = []

    for image_path in image_paths:
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)
        for encoding in encodings:
            known_encodings.append(encoding)
            name = os.path.basename(image_path).split('.')[0]
            known_names.append(name)

    return known_encodings, known_names

def save_encodings(encodings, names, filename):
    with open(filename, 'wb') as f:
        pickle.dump({'encodings': encodings, 'names': names}, f)

if __name__ == "__main__":
    # Provide paths to directories of images for shop owners
    owner_dirs = [
        '/path/to/owner1/',
        '/path/to/owner2/',
        '/path/to/owner3/'
    ]

    image_paths = []
    for owner_dir in owner_dirs:
        for filename in os.listdir(owner_dir):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(owner_dir, filename))

    encodings, names = encode_faces(image_paths)
    save_encodings(encodings, names, 'shop_owners_encodings.pkl')
    print("Shop owners' face encodings saved.")


## real time identification code

In [ ]:
import face_recognition
import pickle
import cv2

def load_encodings(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
        return data['encodings'], data['names']

def identify_face(frame, known_encodings, known_names):
    rgb_frame = frame[:, :, ::-1]  # Convert BGR to RGB
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, encoding)
        name = "Unknown"
        if True in matches:
            first_match_index = matches.index(True)
            name = known_names[first_match_index]

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    return frame

if __name__ == "__main__":
    known_encodings, known_names = load_encodings('shop_owners_encodings.pkl')
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame = identify_face(frame, known_encodings, known_names)
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()
